In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
##https://drive.google.com/file/d/1xP1nXhrI1ZkAC2uezIMYf5-PUUOYYaGQ/view?usp=sharing
your_file = drive.CreateFile({'id':'1xP1nXhrI1ZkAC2uezIMYf5-PUUOYYaGQ'})

In [4]:
your_file.GetContentFile('bitcoin-ta-2015.csv')

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error

# Read dataset
df = pd.read_csv('bitcoin-ta-2015.csv')

# Convert timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Set timestamp as index
df = df.set_index('Timestamp')

# Prepare dataset
train_size = int(len(df) * 0.8)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
print(len(train), len(test))

# Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(train)

# Create training data set
train_data = scaled_data

# Split data into X_train and y_train
X_train = []
y_train = []

for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i])
    y_train.append(train_data[i, 0])

# Convert X_train and y_train to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the data into 3-D array
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

def create_model(params):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=int(params['units']), return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(params['dropout']))
    model.add(Bidirectional(LSTM(units=int(params['units']), return_sequences=False)))
    model.add(Dense(25))
    model.add(Dense(1))
    return model

def objective(params):
    model = create_model(params)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='mean_squared_error')

    # Train the model
    model.fit(params['X_train'], params['y_train'], epochs=int(params['epochs']), batch_size=int(params['batch_size']), validation_split=0.2, verbose=0)


    # Validation
    test_data = scaler.transform(params['test'])
    X_test = []
    y_test = []  # Initialize y_test as empty list

    for i in range(60, len(test_data)):
        X_test.append(test_data[i-60:i, :])  # Appending X_test
        y_test.append(params['test'].iloc[i]['Close'])  # Append corresponding y_test

    y_test = np.array(y_test)  # Convert y_test to numpy array


    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    # Check the model's prediction
    predictions = model.predict(X_test)

    price_scaler = MinMaxScaler(feature_range=(0,1))
    price_scaler.min_, price_scaler.scale_ = scaler.min_[0], scaler.scale_[0]

    # Now you can inverse_transform your predictions
    predictions = price_scaler.inverse_transform(predictions)

    # Calculate mean squared error
    mse = mean_squared_error(y_test, predictions)

    return {'loss': mse, 'status': STATUS_OK}

# Hyperparameters space
space = {
    'units': hp.quniform('units', 50, 200, 1),
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
    'epochs': hp.quniform('epochs', 20, 50, 5),
    'batch_size': hp.quniform('batch_size', 10, 100, 1),
    'X_train': X_train,
    'y_train': y_train,
    'test': test
}

# Trials object to store the optimization history
trials = Trials()

# Run the algorithm
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print(best)

2416 604
17/17 [==============================] - 1s 9ms/step

17/17 [==============================] - 3s 12ms/step

17/17 [==============================] - 1s 8ms/step

17/17 [==============================] - 1s 9ms/step

17/17 [==============================] - 2s 12ms/step

17/17 [==============================] - 1s 8ms/step

17/17 [==============================] - 1s 8ms/step

17/17 [==============================] - 1s 9ms/step

17/17 [==============================] - 1s 10ms/step

17/17 [==============================] - 1s 9ms/step

17/17 [==============================] - 1s 10ms/step

17/17 [==============================] - 1s 8ms/step

17/17 [==============================] - 1s 8ms/step

17/17 [==============================] - 1s 8ms/step

17/17 [==============================] - 2s 11ms/step

17/17 [==============================] - 1s 9ms/step

17/17 [==============================] - 1s 10ms/step

17/17 [==============================] - 2s 9ms/step

17/17 [======